# Create data for mixed-phase clouds


In [ ]:
import numpy as np
import xarray as xr
import glob
import pandas as pd 
import dask

dask.config.set({"array.slicing.split_large_chunks": True}) # this is to avoid memory overload

In [ ]:
# get data
datapath_pd = '/low_clouds_datasets/polar_day/'
datapath_pn = '/low_clouds_datasets/polar_night/'

ds = xr.open_mfdataset(glob.glob(datapath_pd+"LLC_wrapper_tend_*_ICONv1_v6.nc")+
                       glob.glob(datapath_pd+"LLC_meteo*_ICONv1_v6.nc")+
                       glob.glob(datapath_pn+"LLC_wrapper_tend_*_ICONv1_v6.nc")+
                       glob.glob(datapath_pn+"LLC_meteo*_ICONv1_v6.nc"),
                   combine='by_coords', engine='netcdf4', parallel=True, chunks={'time': 'auto'})
# select only levels below 2.5km  
level_idx = 111
ds_mpc = ds.isel(height_2=slice(level_idx,None), height=slice(level_idx,-1))
ds_mpc['W'] = ds_mpc.W.interp(height=ds_mpc.height_2)

# set all non cloud values to nan
ds_mpc = ds_mpc.where((ds_mpc.QC+ds_mpc.QR+ds_mpc.QH+ds_mpc.QI+ds_mpc.QS+ds_mpc.QG)>1e-8, np.nan)

In [ ]:
# select only mixed-phase periods

ds_mpc = ds_mpc.where(( ( ds_mpc.QI+ds_mpc.QS+ds_mpc.QG+ds_mpc.QH) >1e-8) & 
                           (( ds_mpc.QC + ds_mpc.QR) >1e-8), drop=True)
ds_mpc

In [ ]:
# split sublimation and deposition
# store positive values of depsub_tend in a new variable
ds_mpc['deposition_ice_tend']  = ds_mpc['depsub_ice_tend'].where(ds_mpc['depsub_ice_tend'] > 0)
ds_mpc['sublimation_ice_tend'] = ds_mpc['depsub_ice_tend'].where(ds_mpc['depsub_ice_tend'] < 0)
# do same for snow, graupel and hail
ds_mpc['deposition_snow_tend']     = ds_mpc['depsub_snow_tend'].where(ds_mpc['depsub_snow_tend'] > 0)
ds_mpc['sublimation_snow_tend']    = ds_mpc['depsub_snow_tend'].where(ds_mpc['depsub_snow_tend'] < 0)
ds_mpc['deposition_graupel_tend']  = ds_mpc['depsub_graupel_tend'].where(ds_mpc['depsub_graupel_tend'] > 0)
ds_mpc['sublimation_graupel_tend'] = ds_mpc['depsub_graupel_tend'].where(ds_mpc['depsub_graupel_tend'] < 0)
ds_mpc['deposition_hail_tend']     = ds_mpc['depsub_hail_tend'].where(ds_mpc['depsub_hail_tend'] > 0)
ds_mpc['sublimation_hail_tend']    = ds_mpc['depsub_hail_tend'].where(ds_mpc['depsub_hail_tend'] < 0)
ds_mpc['deposition_atmo_tend']     = ds_mpc['depsub_atmo_tend'].where(ds_mpc['depsub_atmo_tend'] > 0)
ds_mpc['sublimation_atmo_tend']    = ds_mpc['depsub_atmo_tend'].where(ds_mpc['depsub_atmo_tend'] < 0)

# split condensation and evaporation
# store positive values of satad_cloud_tend in a new variable
ds_mpc['condensation_cloud_tend'] = ds_mpc['satad_cloud_tend'].where(ds_mpc['satad_cloud_tend'] > 0)
ds_mpc['evaporation_cloud_tend']  = ds_mpc['satad_cloud_tend'].where(ds_mpc['satad_cloud_tend'] < 0)
ds_mpc['condensation_atmo_tend']  = ds_mpc['satad_atmo_tend'].where(ds_mpc['satad_atmo_tend'] > 0)
ds_mpc['evaporation_atmo_tend']   = ds_mpc['satad_atmo_tend'].where(ds_mpc['satad_atmo_tend'] < 0)


In [ ]:
# get all tendencies which mention ice, snow, graupel or hail and not total
frozen_tend = [tend for tend in ds_mpc.data_vars if ('ice' in tend or 'snow' in tend or 'graupel' in tend \
                                                        or 'hail' in tend) and 'total' not in tend \
                                                        and 'depsub' not in tend ]
print(frozen_tend)
# compute sum over all tendencies which mention ice, snow, graupel or hail
#ds_mpc['total_mass_tend']= ds_mpc[frozen_tend].to_array().sum(dim='variable')
# same for liquid water so with rain and cloud
liquid_tend = [tend for tend in ds_mpc.data_vars if ('rain' in tend or 'cloud' in tend) \
                                                       and 'total' not in tend and 'satad' not in tend]
print(liquid_tend)

atmo_tend = [tend for tend in ds_mpc.data_vars if ('atmo' in tend) and 'total' not in tend \
                                 and 'satad' not in tend and 'depsub' not in tend]
print(atmo_tend)

In [ ]:

#ds_mpc['total_mass_tend'] = ds_mpc[liquid_tend].to_array().sum(dim='variable')
# frozen processes
deposition_fr_vars = [var for var in frozen_tend if 'deposition' in var ]
sublimation_fr_vars = [var for var in frozen_tend if 'sublimation' in var ]
rime_fr_vars = [var for var in frozen_tend if 'rime' in var ]
homhet_fr_vars = [var for var in frozen_tend if 'homhet' in var ]
c_homfr_fr_vars = [var for var in frozen_tend if 'c_homfr' in var ]
fr_col_fr_vars = [var for var in frozen_tend if 'col_' in var]
fr_eva_fr_vars = [var for var in frozen_tend if 'eva' in var ]
g_to_h_fr_vars = [var for var in frozen_tend if 'g_to_h' in var  ]
r_freeze_fr_vars = [var for var in frozen_tend if 'r_freeze' in var ]
melt_fr_vars = [var for var in frozen_tend if 'melt' in var ]

# liquid processes
au_li_vars= [var for var in liquid_tend if 'auSB' in var ]
ac_li_vars = [var for var in liquid_tend if 'acSB' in var ]
eva_li_vars = [var for var in liquid_tend if 'eva' in var ] # rain evaporation and cloud evaporation both (so includes satad)
c_homfr_li_vars = [var for var in liquid_tend if 'homfr' in var ]
homhet_li_vars = [var for var in liquid_tend if 'homhet' in var ]
rime_li_vars = [var for var in liquid_tend if 'rime' in var ]
r_freeze_li_vars = [var for var in liquid_tend if 'r_freeze' in var ]
melt_li_vars = [var for var in liquid_tend if 'melt' in var ]
cond_li_vars = [var for var in liquid_tend if 'condensation' in var ]
ccn_act_li_vars = [var for var in liquid_tend if 'ccn_act' in var ]

In [ ]:
# togethe in a pandas dataframe
df_all_tends = pd.DataFrame(ds_mpc[deposition_fr_vars].to_array().sum(dim="variable").values.flatten(),
                            columns=['deposition_fr'])
#df_fr_tends['deposition'] = ds_mpc[deposition_fr_vars].to_array().sum(dim="variable").values[total_mass > 1e-8].flatten()
df_all_tends['sublimation_fr'] = np.abs(ds_mpc[sublimation_fr_vars].to_array().sum(dim="variable").values).flatten()
df_all_tends['rime_fr'] = ds_mpc[rime_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['homhet_fr'] = ds_mpc[homhet_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['c_homfr_fr'] = ds_mpc[c_homfr_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['fr_col'] = ds_mpc[fr_col_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['fr_eva_fr'] = np.abs(ds_mpc[fr_eva_fr_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['g_to_h'] = ds_mpc[g_to_h_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['r_freeze_fr'] = ds_mpc[r_freeze_fr_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['melt_fr'] = np.abs(ds_mpc[melt_fr_vars].to_array().sum(dim="variable").values.flatten())

# compute sum over all tendencies which mention ice, snow, graupel or hail
df_all_tends['total_fr'] = df_all_tends.sum(axis=1)
# for liquid water
df_all_tends["auSB"] = ds_mpc[au_li_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['acSB'] = ds_mpc[ac_li_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['evaporation_li'] = np.abs(ds_mpc[eva_li_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['c_homfr_li'] = np.abs(ds_mpc[c_homfr_li_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['homhet_li'] = np.abs(ds_mpc[homhet_li_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['rime_li'] = np.abs(ds_mpc[rime_li_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['r_freeze_li'] = np.abs(ds_mpc[r_freeze_li_vars].to_array().sum(dim="variable").values.flatten())
df_all_tends['melt_li'] = ds_mpc[melt_li_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['condensation_li'] = ds_mpc[cond_li_vars].to_array().sum(dim="variable").values.flatten()
df_all_tends['ccn_act_li'] = ds_mpc[ccn_act_li_vars].to_array().sum(dim="variable").values.flatten()

# total tendency                                                            
#df_li_tends['total_li'] = df_li_tends.sum(axis=1)


In [ ]:
# add masses to dataframe
df_all_tends['QI'] = ds_mpc.QI.values.flatten()
df_all_tends['QS'] = ds_mpc.QS.values.flatten()
df_all_tends['QG'] = ds_mpc.QG.values.flatten()
df_all_tends['QH'] = ds_mpc.QH.values.flatten()
df_all_tends['QR'] = ds_mpc.QR.values.flatten()
df_all_tends['QC'] = ds_mpc.QC.values.flatten()
df_all_tends['QV'] = ds_mpc.QV.values.flatten()
df_all_tends['frozen mass'] = ds_mpc.QI.values.flatten()+ds_mpc.QS.values.flatten()+ds_mpc.QG.values.flatten()+ds_mpc.QH.values.flatten()
df_all_tends['liquid mass'] = ds_mpc.QR.values.flatten()+ds_mpc.QC.values.flatten()


In [ ]:
display(df_all_tends.describe()) 

In [ ]:
df_all_tends.to_csv('../../../data/processed/df_all_tends_mpc_pn_and_pd.csv')